<a href="https://colab.research.google.com/github/joaopedropastura/credit_score_ai/blob/main/v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn
!pip install pandas
!pip install joblib


In [52]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from joblib import load, dump
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.kernel_approximation import RBFSampler


In [169]:
path = "/content/credit_score.csv"

source = pd.read_csv(path)
le = LabelEncoder()

<ipython-input-169-0f7882e52b77>:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  source = pd.read_csv(path)


In [153]:

source = source[source['Payment_Behaviour'] != '!@9#%8']


In [174]:
sla = source['Type_of_Loan'].unique()

sla = source[source['Type_of_Loan'] != 'Not Specified' || ]

sla

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,Standard


In [154]:
def convert_to_months(age):
    if type(age) == type('String'):
      age_split = age.split(' ')
      years = int(age_split[0])
      months = int(age_split[3])
      # Cálculo total em meses
      total_months = years * 12 + months
      return total_months


source['Credit_History_Age'] = source['Credit_History_Age'].apply(convert_to_months)


In [155]:
source = source.drop(['ID', 'Customer_ID', 'SSN', 'Name', 'Type_of_Loan'], axis=1)

In [156]:
source = source.head(3000)


In [157]:
Month = source["Month"].unique()
Credit_History_Age = source['Credit_History_Age'].unique()
Occupation = source["Occupation"].unique()
Credit_Mix = source["Credit_Mix"].unique()
Credit_Score = source["Credit_Score"].unique()
Payment_Behaviour = source['Payment_Behaviour'].unique()
Payment_of_Min_Amount = source["Payment_of_Min_Amount"].unique()

In [ ]:
le.fit(Month)
source['Month'] = le.transform(source['Month'])

le.fit(Payment_of_Min_Amount)
source['Payment_of_Min_Amount'] = le.transform(source['Payment_of_Min_Amount'])

le.fit(Credit_History_Age)
source['Credit_History_Age'] = le.transform(source['Credit_History_Age'])

le.fit(Occupation)
source['Occupation'] = le.transform(source['Occupation'])

le.fit(Credit_Mix)
source['Credit_Mix'] = le.transform(source['Credit_Mix'])

le.fit(Credit_Score)
source['Credit_Score'] = le.transform(source['Credit_Score'])

le.fit(Payment_Behaviour)
source['Payment_Behaviour'] = le.transform(source['Payment_Behaviour'])

In [160]:
source["Annual_Income"] = source["Annual_Income"].str[:-1]

In [161]:
def filterAge(age):
  try:
    pattern = r'\b\d+(?:[-_]\d+)*\b\s+'
    number = re.findall(pattern, age)
    return ' '.join(number)
  except:
    return age


for elem in source:

  source[elem] = source[elem].apply(filterAge)



In [162]:
for elem in source:
  source[elem].replace('', 0, inplace=True)

source = source.fillna(0)

In [114]:
# for column in source:
#   source[column] = source[column] /source[column].abs().max()



# source
# source = source.fillna(0)

In [127]:
source


,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,3,0,12,0,1824.843333,3,4,3,0,3,...,4.0,3,0,26.822620,1,49.574949,0.0,2,0.0,0
1,2,0,12,0,0.000000,3,4,3,0,-1,...,4.0,1,0,31.944960,1,49.574949,0.0,3,0.0,0
2,6,0,12,0,0.000000,3,4,3,0,3,...,4.0,1,0,28.609352,1,49.574949,0.0,4,0.0,0
3,0,0,12,0,0.000000,3,4,3,0,5,...,4.0,1,0,31.377862,1,49.574949,0.0,5,0.0,0
4,7,0,12,0,1824.843333,3,4,3,0,6,...,4.0,1,0,24.797347,1,49.574949,0.0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,1,0,9,0,11587.810000,5,5,13,0,10,...,6.0,2,0,24.483625,2,385.601185,0.0,1,0.0,0
3240,3,0,8,0,0.000000,5,5,5,0,10,...,7.0,2,0,23.428472,2,60.108482,0.0,4,0.0,2
3241,2,0,8,0,4300.731667,5,5,5,0,14,...,12.0,2,0,35.641549,2,60.108482,0.0,1,0.0,2
3242,6,0,8,0,4300.731667,5,5,5,0,14,...,12.0,2,0,37.045047,2,29308.000000,0.0,1,0.0,2


In [163]:
X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

X_train, X_test, Y_train, Y_test = \
  train_test_split(X, Y, test_size=0.2, random_state=50)

In [164]:
#RANDOM FOREST COM PCA

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = DecisionTreeClassifier(max_depth=100, min_samples_split=36)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


0.6783333333333333
0.8020833333333334


In [165]:
#RANDOM FOREST COM PCA


X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = RandomForestClassifier(max_depth=100, min_samples_split=36)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


0.7383333333333333
0.8308333333333333


In [166]:
#SVM COM PCA

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


model = SVC()

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


0.54
0.5304166666666666


In [167]:
#NUSVC COM PCA

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

# Aplicação do PCA apenas nos dados de treinamento
pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


model = make_pipeline(StandardScaler(), NuSVC())

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)


0.5416666666666666
0.71


In [133]:
#SGD

X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = accuracy_score(Y_test, Y_pred)
accurac3y = accuracy_score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)

0.32166666666666666
0.345


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [ ]:
#Kernel Approximation


X = source.drop('Credit_Score', axis=1)
Y = source['Credit_Score']

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=500)

pca = PCA(n_components=20)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X_train)

model = SGDClassifier(max_iter=5)


model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)
accuracy = model.score(Y_test, Y_pred)
accurac3y = model.score(Y_train, Y_pred_train)

matrix = confusion_matrix(Y_test, Y_pred)

print(accuracy)
print(accurac3y)